In [ ]:
from Go2Py.robot.fsm import FSM
from Go2Py.robot.remote import KeyboardRemote, XBoxRemote
from Go2Py.robot.safety import SafetyHypervisor
from Go2Py.sim.mujoco import Go2Sim
from Go2Py.control.cat import *
import torch

In [ ]:
# load the ppo controller
import jax
import functools
from brax.training.agents.ppo import networks as ppo_networks
from brax.training.acme import running_statistics
import jax.numpy as jp
import brax.io as bio
from brax.io import  model as bmodel
from brax import math


#policy_path = "mjx_brax_quadruped_policy_245760"
policy_path = "qpolicy_fs_03_params"
params = bmodel.load_params(policy_path)
# print lets load the new params!
#my_inference_fn = make_inference_fn((params[0], params[1].policy))  
# identity = lambda x, y: x

import mujoco

normalize = running_statistics.normalize

obs_size = 15 * 31
action_size = 12

_ppo_networks =  ppo_networks.make_ppo_networks(
    observation_size = obs_size,
    action_size = 12,
    preprocess_observations_fn = normalize,
    policy_hidden_layer_sizes=(128, 128, 128, 128)) 

my_inference_fn = ppo_networks.make_inference_fn(_ppo_networks)((params[0], params[1].policy))
fake_obs = jp.zeros(15*31)
prng_key = jax.random.PRNGKey(0)
out = my_inference_fn(fake_obs, prng_key)
print(out)







In [3]:
def get_obs_v2(
    mj_model, 
    mj_data,
    joystick_cmd: jp.ndarray,
    last_act: jp.ndarray,
    default_pose: jp.ndarray,
    q : jp.ndarray,
) -> jp.ndarray:

   
    id =  mj_model.body(name='base').id
    print("xpos" , mj_data.xpos[id])
    print("xquat", mj_data.xquat[id])

   
    inv_torso_rot = math.quat_inv(mj_data.xquat[id])
    local_rpyrate = math.rotate(mj_data.cvel[id][:3], inv_torso_rot)

    obs = jp.concatenate([
        jp.array([local_rpyrate[2]]) * 0.25,                 # yaw rate
        math.rotate(jp.array([0, 0, -1]), inv_torso_rot),    # projected gravity
        joystick_cmd * jp.array([2.0, 2.0, 0.25]),  # command
        q - default_pose,           # motor angles
        last_act                              # last action
    ])

    return obs

In [ ]:
from mujoco import viewer
import mujoco

spec = mujoco.MjSpec.from_file( "../Go2Py/assets/mujoco/scene_mjx.xml")

spec.option.timestep = 0.004


for j in spec.joints[1:]:
    j.damping = 0.5239

# for a in spec.actuators:
#     a.gainprm[0] = 35.0 
#     a.biasprm[1] = -35.0

for a in spec.actuators:
    a.gainprm[0] = 45.0 
    a.biasprm[1] = -45.0

mj_model = spec.compile()
mj_data = mujoco.MjData(mj_model)
mj_data.qpos = mj_model.keyframe("home").qpos
mj_data.qvel = np.zeros(12 + 6)


breakpoint()
n_steps = 1000
joystick_cmd = np.zeros(3)
qpos= np.array([0, 0, 0.27, 1, 0, 0, 0 ,0 ,0.9 ,-1.8, 0 ,0.9, -1.8 ,0 ,0.9 ,-1.8 ,0 ,0.9 ,-1.8])
default_pose = qpos[7:]
obs_dim = 31
render_every = 5
last_ctrl = np.zeros(12)
obs_history = np.zeros(15 * 31)
joystick_cmd = np.array([0, 0 , 0])

with viewer.launch_passive(mj_model, mj_data) as t_viewer:

    for i in range(10* n_steps):

        # Extract observations
        # observation = get_observation(mj_model, mj_data)
        obs = get_obs_v2(mj_model, mj_data, jp.array(joystick_cmd), jp.array(last_ctrl), default_pose)
        print("current obs is " , obs)
        obs_history = np.roll(obs_history, obs_dim)
        obs_history[:obs_dim] = obs  

        # normalize obs
        # obs_normlized = (obs_history - obs_history_mean) / obs_history_std

        # evalaute the policy
        _obs_history = jp.array(obs_history)
        ctrl = my_inference_fn(_obs_history, prng_key)[0]
        last_ctrl = ctrl
        print("ctrl is", ctrl)
        

        mj_data.ctrl = default_pose  + .3 * ctrl
        mj_data.ctrl = default_pose 

        # Step the simulation
        for _ in range(5):
            mujoco.mj_step(mj_model, mj_data)

        mujoco.mj_forward(mj_model, mj_data)

        t_viewer.sync()
        # t_viewer.sync()

        time.sleep(.02)

        # # Render every 'render_every' steps
        # if i % render_every == 0:
        #     renderer.update_scene(mj_data)
        #     images.append(renderer.render())

        # # Optional: Print progress
        # if i % 10 == 0:
        #     print(f"Step {i}/{n_steps}")


In [ ]:
import mujoco
from mujoco import viewer
spec = mujoco.MjSpec.from_file( "../Go2Py/assets/mujoco/scene_mjx_torque.xml")
#spec = mujoco.MjSpec.from_file( "../Go2Py/assets/mujoco/go2.xml")


spec.option.timestep = 0.001


for j in spec.joints[1:]:
    j.damping = 0.5239

# for a in spec.actuators:
#     a.gainprm[0] = 35.0 
#     a.biasprm[1] = -35.0



mj_model = spec.compile()
mj_data = mujoco.MjData(mj_model)
mj_data.qpos = mj_model.keyframe("home").qpos
mj_data.qvel = np.zeros(12 + 6)

kp = 50.
kd = 1.


n_steps = 1000
joystick_cmd = np.zeros(3)
qpos= mj_model.keyframe("home").qpos
default_pose = qpos[7:]
obs_dim = 31
render_every = 5
last_ctrl = np.zeros(12)
obs_history = np.zeros(15 * 31)
joystick_cmd = np.array([0., 0., 0.])

with viewer.launch_passive(mj_model, mj_data) as t_viewer:

    for i in range( 10* n_steps):

        # Extract observations
        # observation = get_observation(mj_model, mj_data)
        obs = get_obs_v2(mj_model, mj_data, jp.array(joystick_cmd), jp.array(last_ctrl), default_pose)
        #print("current obs is " , obs)
        obs_history = np.roll(obs_history, obs_dim)
        obs_history[:obs_dim] = obs  

        # normalize obs
        # obs_normlized = (obs_history - obs_history_mean) / obs_history_std

        # evalaute the policy
        _obs_history = jp.array(obs_history)
        ctrl = my_inference_fn(_obs_history, prng_key)[0]
        last_ctrl = ctrl
        #print("ctrl is", ctrl)
        

        qdes =  default_pose  + .3 * ctrl

        # Step the simulation
        for _ in range(5):
            q = mj_data.qpos[7:]
            dq = mj_data.qvel[6:]
            qdes = default_pose
            print("q",q)
            print("qdes",qdes)
            print("qdes - q")
            print(qdes - q )
            mj_data.ctrl = kp * (qdes - q) - kd * dq
            #mj_data.ctrl = np.clip(mj_data.ctrl, -35, 35)
            mujoco.mj_step(mj_model, mj_data)
            time.sleep(.002)

        #mujoco.mj_forward(mj_model, mj_data)

        t_viewer.sync()
        # t_viewer.sync()

        #time.sleep(2.)

        # # Render every 'render_every' steps
        # if i % render_every == 0:
        #     renderer.update_scene(mj_data)
        #     images.append(renderer.render())

        # # Optional: Print progress
        # if i % 10 == 0:
        #     print(f"Step {i}/{n_steps}")

In [ ]:
from Go2Py.sim.mujoco import Go2Sim
from Go2Py.robot.model import Go2Model
import mujoco

from Go2Py.robot.model import FrictionModel
import numpy as np
from copy import deepcopy
from brax import math
#friction_model = FrictionModel(Fs=3, mu_v=0.05)
friction_model = None
robot = Go2Sim(dt = 0.001, friction_model=friction_model,
               camera_name = "track",
                    xml_path="../Go2Py/assets/mujoco/scene_mjx.xml"
               )

# robot.standUpReset()
running = True
# kp = 35 * np.ones(12)
# kd = 0.5 * np.ones(12)
# tau_ff = np.zeros(12)
# dq_des = np.zeros(12)
# q_des = np.zeros(12)



qpos = np.array([0, 0, 0.27, 1, 0, 0, 0 ,0 ,0.9 ,-1.8, 0 ,0.9, -1.8 ,0 ,0.9 ,-1.8 ,0 ,0.9 ,-1.8])

joystick_cmd = np.zeros(3)
       
        
robot.data.qpos = qpos
robot.data.qvel = np.zeros(18)
robot.data.ctrl = qpos[7:]
mujoco.mj_forward(robot.model, robot.data)
if robot.render:
    robot.viewer.sync()

joint_state = robot.getJointStates()
print(joint_state)
q_des =deepcopy(joint_state['q'])
       



obs_history = np.zeros(31*15)

last_ctrl = np.zeros(12)
default_pose = qpos[7:]
action_scale = .3
obs_dim = 31
# joystick_cmd = np.array([0., 0. , 0.])
# for i in range(15):
#     obs = get_obs_v2(robot.model, robot.data,  jp.array(joystick_cmd), jp.array(last_ctrl), default_pose)
#     obs_history = np.roll(obs_history, obs_dim)
#     obs_history[:obs_dim] = obs  



joystick_cmd = np.array([1., 0, 1.5])
while running:
    joint_state = robot.getJointStates()

    obs = get_obs_v2(robot.model, robot.data,  jp.array(joystick_cmd), jp.array(last_ctrl), default_pose)
    
    obs_history = np.roll(obs_history, obs_dim)
    obs_history[:obs_dim] = obs  

    _obs_history = jp.array(obs_history)
    ctrl = my_inference_fn(_obs_history, prng_key)[0]
    
    last_ctrl = ctrl
    print("ctrl is", ctrl)
  

    q_des = default_pose  + action_scale * ctrl
    robot.data.ctrl = q_des

    #robot.setCommands(q_des, dq_des, kp, kd, tau_ff)
    for _ in range(20):
        mujoco.mj_step(robot.model, robot.data)
        time.sleep(0.001)
    mujoco.mj_forward(robot.model, robot.data)
    robot.viewer.sync()

#TODO: apply this to the one from roohalla.





In [4]:
# now lets use the normal SIM of the robot
from Go2Py.sim.mujoco import Go2Sim
from Go2Py.robot.model import Go2Model
import mujoco

from Go2Py.robot.model import FrictionModel
import numpy as np
from copy import deepcopy
from brax import math
#friction_model = FrictionModel(Fs=3, mu_v=0.05)
#friction_model = FrictionModel(Fs=0., mu_v=2.)
#friction_model = None
friction_model = FrictionModel(Fs=3., mu_v=0.05)
robot = Go2Sim(dt = 0.001, friction_model=friction_model,
               xml_path="../Go2Py/assets/mujoco/go2.xml",
              # camera_name = "track", 
                #  camera_resolution = (640, 480),
                #  camera_depth_range = (0.35, 3.0), 
                 
               
               )
               


In [ ]:

# robot.standUpReset()
running = True
kp = 20. * np.ones(12)
kd = .5 * np.ones(12)
tau_ff = np.zeros(12)
dq_des = np.zeros(12)
q_des = np.zeros(12)



qpos= np.array([0, 0, 0.27, 1, 0, 0, 0 ,0 ,0.9 ,-1.8, 0 ,0.9, -1.8 ,0 ,0.9 ,-1.8 ,0 ,0.9 ,-1.8])

joystick_cmd = np.zeros(3)
       
        
robot.data.qpos = qpos
robot.data.qvel = np.zeros(18)

mujoco.mj_forward(robot.model, robot.data)
if robot.render:
    robot.viewer.sync()

joint_state = robot.getJointStates()
print(joint_state)
q_des =deepcopy(joint_state['q'])
       



obs_history = np.zeros(31*15)

last_ctrl = np.zeros(12)
default_pose = qpos[7:]
action_scale = .3
action_scale = .5 #new one!


obs_dim = 31
joystick_cmd = np.array([0., 0. , 0.])
q = q_des
for i in range(15):
    obs = get_obs_v2(robot.model, robot.data,  jp.array(joystick_cmd), jp.array(last_ctrl), default_pose,q)
    obs_history = np.roll(obs_history, obs_dim)
    obs_history[:obs_dim] = obs  



joystick_cmd = np.array([1.,0., 0.5])
counter = 0


In [ ]:
# roohala:
# FR_hip 
# FL_hip
# RR_hip, 
# RL_hip

# quim:
# FL_hip
# FR_hip
# RL_hip
# RR_hip

def roh_to_q(q):
    # switch the order of joints.
    # 0,1,2 -> 0,1,2
    # 0,1,2 -> 
    assert ( len(q) == 12)
    out = np.copy(q)
    out[:3] = q[3:6]
    out[3:6] = q[:3]
    out[6:9] = q[9:12]
    out[9:12] = q[6:9]
    return out

def roh_to_q(q):
    return q

def q_to_roh(q):
    assert ( len(q) == 12)
    return roh_to_q(q)

def q_to_roh(q):
    return q


counter = 0
while running:
    joint_state = robot.getJointStates()

    q = roh_to_q(joint_state['q'])

    obs = get_obs_v2(robot.model, robot.data,  jp.array(joystick_cmd), jp.array(last_ctrl), default_pose,
                     q)
    
    obs_history = np.roll(obs_history, obs_dim)
    obs_history[:obs_dim] = obs  

    _obs_history = jp.array(obs_history)
    ctrl = my_inference_fn(_obs_history, prng_key)[0]
    
    last_ctrl = ctrl
    # print("ctrl is", ctrl)
  

    q_des = default_pose  + action_scale * ctrl

    # print("q_des", q_des)
    # print("q_current", joint_state['q'])

    #q_des_roh = q_to_roh(np.array(q_des))
    q_des_roh = q_des
    robot.setCommands(q_des_roh, dq_des, kp, kd, tau_ff)

    #robot.setCommands(q_des, dq_des, kp, kd, tau_ff)
    for _ in range(20):
        robot.stepLowlevel()
        #mujoco.mj_step(robot.model, robot.data)
        time.sleep(0.001)
    mujoco.mj_forward(robot.model, robot.data)
    robot.viewer.sync()
    counter += 1
    #nput('press to continue')
    if counter > 1000:
        break

#TODO: apply this to the one from roohalla.